In [5]:
import matplotlib.pyplot as plt
plt.rcParams["savefig.bbox"] = 'tight'

import torch
from torch import nn
import torchvision
import torchvision.transforms as T
import torchvision.transforms.functional as TF

import lightning.pytorch as pl
import os

In [6]:
# sets seeds for numpy, torch and python.random.
pl.seed_everything(42, workers=True)

Global seed set to 42


42

# Transforms

In [10]:
train_transform = T.Compose([
    T.Resize((132, 132)),
    T.RandomCrop((128, 128)),
    T.RandomAdjustSharpness(sharpness_factor=4)
    T.RandomPerspective(distortion_scale=0.12)
    T.ColorJitter(brightness=0.1, hue=0.1, contrast=0.1, saturation=0.1),
    T.GaussianBlur(kernel_size=3),
    T.ToTensor()
])

In [11]:
test_transform = T.Compose([
    T.Resize((128, 128)),
    T.ToTensor()
])

# Data Module

In [12]:
from data_module import TripletDataset, UdacitySimDataModule

NUM_WORKERS = os.cpu_count() # <- use all available CPU cores

udacity_sim_dm = UdacitySimDataModule(csv_path="\data\driving_log.csv",
                                      images_dir="data\IMG",
                                      batch_size=32,
                                      num_workers=NUM_WORKERS,
                                      train_transform=train_transform,
                                      test_transform=test_transform)

# PyTorch Model

In [13]:
from model import BabyHamiltonModel
model = BabyHamiltonModel()

# Lightning Model

In [14]:
from train import LightingBabyHamiltonModel
lightning_model = LightingBabyHamiltonModel(model)

# Callbacks

In [16]:
from train import get_callbacks, get_logger

callbacks = get_callbacks()
logger = get_logger()

# Trainer

In [17]:
trainer = pl.Trainer(
    # Permanent
    default_root_dir="checkpoints/",
    callbacks=callbacks,
    max_epochs=10,
    devices='auto', # 1, 2, ...
    accelerator="auto", # gpu, cpu, ...
    logger=logger,
    check_val_every_n_epoch=1,
    
    # Temporary
    
    fast_dev_run=True,
    profiler="simple",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


# Tuner

In [18]:
from lightning.pytorch.tuner import Tuner
tuner = Tuner(trainer)

## Learning Rate

In [ ]:
lr_finder = tuner.lr_find(
    lightning_model, 
    datamodule=udacity_sim_dm
)

In [ ]:
lr_finder.plot(suggest=True)

In [ ]:
new_lr = lr_finder.suggestion()
lightning_model.learning_rate = new_lr
new_lr

## Batch Size

In [ ]:
bs_finder = tuner.scale_batch_size(
    lightning_model, 
    datamodule=udacity_sim_dm
)

In [ ]:
bs_finder.plot(suggest=True)

In [ ]:
new_bs = bs_finder.suggestion()
udacity_sim_dm.batch_size = new_bs
lightning_model.batch_size = new_bs
new_bs

# Training

## Fit

In [ ]:
trainer.fit(
    lightning_model, 
    datamodule=udacity_sim_dm
)

## Plot Metrics

In [ ]:
from utils import plot_csv_logger
plot_csv_logger(csv_path=f'{trainer.logger.log_dir}/metrics.csv')

# Testing

In [ ]:
trainer.test(
    lightning_model, 
    datamodule=udacity_sim_dm, 
    ckpt_path='best'
)

# Production

In [ ]:
path = trainer.checkpoint_callback.best_model_path
lightning_model = LightningBabyHamiltonModel.load_from_checkpoint(path)

In [ ]:
x = torch.randn(1, 64)

with torch.inference_mode():
    y_hat = lightning_model(x)